In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib as mpl
import re
mpl.rcParams['font.family'] = 'serif'
import os

In [2]:
current_folder = globals()['_dh'][0]
rootdir = os.path.dirname(os.path.dirname(current_folder))
indir_inter = os.path.join(rootdir, '_2_intermediate', 'data')
indir_prepro = os.path.join(rootdir, '_1_preprocessing', 'data', 'raw', 'gis')
outdir = os.path.join(rootdir, '_3_figures_tables', 'data')

In [3]:
data = pd.read_csv(os.path.join(indir_inter, '_F_dist_religion_bch10.csv'))[['iso',
                                                         'bch10',
                                                         'major_religion',
                                                         'district',
                                                         'immg_18',
                                                         'n_immg_18']]
data = data[data.major_religion.isin(['Christian', 'Muslim', 'Traditional'])].reset_index(drop=True)
data = data[~((pd.isnull(data.immg_18)) & (pd.isnull(data.n_immg_18)))]

#####################################################################################

data1980 = data[data.bch10.isin([1980, 1990])].reset_index(drop=True)
data1980 = data1980[~((data1980.iso.isin(['LBR','MLI','NGA','TGO'])) & 
                      (data1980.bch10==1980))].reset_index(drop=True)
data1980 = data1980[~((~(data1980.iso.isin(['LBR','MLI','NGA','TGO']))) & 
                      (data1980.bch10==1990))].reset_index(drop=True)
del data1980['bch10']

data1980.index = [data1980.iso, data1980.district, data1980.major_religion]
del data1980['iso'], data1980['district'], data1980['major_religion']
data1980 = data1980.unstack().reset_index(drop=False)
data1980.columns = [x[0] + x[1] for x in data1980.columns]

data1980['gap_cm'] = data1980.immg_18Christian - data1980.immg_18Muslim
data1980['gap_ct'] = data1980.immg_18Christian - data1980.immg_18Traditional

data1980['gap_cm_min10'] = data1980['gap_cm']
data1980['gap_ct_min10'] = data1980['gap_ct']
data1980.loc[(data1980.n_immg_18Christian < 10) | (data1980.n_immg_18Muslim < 10), 'gap_cm_min10'] = np.NaN
data1980.loc[(data1980.n_immg_18Christian < 10) | (data1980.n_immg_18Traditional < 10), 'gap_ct_min10'] = np.NaN
data1980 = data1980[['iso', 'district', 'gap_cm', 'gap_ct', 'gap_cm_min10', 'gap_ct_min10']]

data1980 = data1980[~((pd.isnull(data1980.gap_cm)) &
                      (pd.isnull(data1980.gap_ct)) &
                      (pd.isnull(data1980.gap_cm_min10)) &
                      (pd.isnull(data1980.gap_ct_min10)))]

cns = ['iso', 'district'] + [x + '_1980' for x in data1980.columns[2:]]
data1980.columns = cns
del data1980['iso']


#####################################################################################

data['x'] = data.immg_18 * data.n_immg_18
del data['bch10']
data = data.groupby([data.iso, data.district, data.major_religion]).sum()
# del data['district']
data = data.reset_index(drop=False)
data['immg_18'] = data['x'] / data['n_immg_18']
del data['x']

data.index = [data.iso, data.district, data.major_religion]
del data['iso'], data['district'], data['major_religion']
data = data.unstack().reset_index(drop=False)
data.columns = [x[0] + x[1] for x in data.columns]

data['gap_cm'] = data.immg_18Christian - data.immg_18Muslim
data['gap_ct'] = data.immg_18Christian - data.immg_18Traditional

data['gap_cm_min10'] = data['gap_cm']
data['gap_ct_min10'] = data['gap_ct']
data.loc[(data.n_immg_18Christian < 10) | (data.n_immg_18Muslim < 10), 'gap_cm_min10'] = np.NaN
data.loc[(data.n_immg_18Christian < 10) | (data.n_immg_18Traditional < 10), 'gap_ct_min10'] = np.NaN
data = data[['iso', 'district', 'gap_cm', 'gap_ct', 'gap_cm_min10', 'gap_ct_min10']]

data = data[~((pd.isnull(data.gap_cm)) &
              (pd.isnull(data.gap_ct)) &
              (pd.isnull(data.gap_cm_min10)) &
              (pd.isnull(data.gap_ct_min10)))]

In [4]:
data1980

,district,gap_cm_1980,gap_ct_1980,gap_cm_min10_1980,gap_ct_min10_1980
0,1.0,0.009954,0.066924,0.009954,0.066924
1,2.0,0.141636,-0.120567,0.141636,NaN
2,3.0,-0.020984,0.143885,-0.020984,NaN
3,4.0,0.935103,1.000000,NaN,NaN
4,5.0,0.605590,NaN,0.605590,NaN
...,...,...,...,...,...
2258,2753.0,NaN,0.053374,NaN,0.053374
2259,2754.0,NaN,0.068489,NaN,0.068489
2260,2755.0,NaN,-0.023202,NaN,-0.023202
2261,2756.0,NaN,-0.160833,NaN,-0.160833


In [5]:
data1980[~pd.isnull(data1980.gap_cm_1980)]

,district,gap_cm_1980,gap_ct_1980,gap_cm_min10_1980,gap_ct_min10_1980
0,1.0,0.009954,0.066924,0.009954,0.066924
1,2.0,0.141636,-0.120567,0.141636,NaN
2,3.0,-0.020984,0.143885,-0.020984,NaN
3,4.0,0.935103,1.000000,NaN,NaN
4,5.0,0.605590,NaN,0.605590,NaN
...,...,...,...,...,...
2250,2745.0,-0.623256,0.013108,NaN,0.013108
2251,2746.0,-0.557377,-0.057377,NaN,-0.057377
2253,2748.0,0.495506,-0.060050,NaN,-0.060050
2254,2749.0,0.621813,0.121813,NaN,NaN


In [6]:
ISO = sorted(list(set(data.iso)))

In [7]:
countries = gpd.read_file(os.path.join(indir_prepro, 'africa_countries', 'africa_countries.shp'))
afrregions = gpd.read_file(os.path.join(indir_prepro, 'districts_for_plotting', 'districts_for_plotting.shp'))[['iso', 'district', 'geometry']]
afrregions = afrregions[afrregions.iso.isin(ISO)].reset_index(drop=True).copy()
afrregions = pd.merge(afrregions, data, on=['iso', 'district'], how='left')
afrregions = pd.merge(afrregions, data1980, on='district', how='left')
gha = afrregions[(afrregions.geometry!=None) & (afrregions.iso == 'GHA')].reset_index(drop=True).copy(deep=True)
gha = gha[['iso', 'district', 'geometry', 'gap_cm', 'gap_ct', 'gap_cm_min10', 'gap_ct_min10']]
# ghac = countries[countries.iso3 == 'GHA'].copy(deep=True)

In [8]:
gha

,iso,district,geometry,gap_cm,gap_ct,gap_cm_min10,gap_ct_min10
0,GHA,680.0,"POLYGON ((-2.60876 5.41339, -2.60723 5.41003, ...",0.118025,-0.002550,0.118025,-0.002550
1,GHA,681.0,"MULTIPOLYGON (((-2.24836 4.85622, -2.24917 4.8...",-0.054675,0.377886,-0.054675,0.377886
2,GHA,682.0,"MULTIPOLYGON (((-1.93167 4.79789, -1.93339 4.7...",0.090380,0.178616,0.090380,0.178616
3,GHA,683.0,"POLYGON ((-1.58169 5.15213, -1.58003 5.13779, ...",0.003917,0.421117,0.003917,0.421117
4,GHA,684.0,"POLYGON ((-1.44765 5.21947, -1.44558 5.21375, ...",0.112562,0.203186,0.112562,0.203186
...,...,...,...,...,...,...,...
105,GHA,785.0,"POLYGON ((-1.87033 10.37888, -1.86873 10.37741...",-0.078621,0.248969,-0.078621,0.248969
106,GHA,786.0,"POLYGON ((-2.31801 10.57895, -2.31769 10.57885...",0.066182,0.313025,0.066182,0.313025
107,GHA,787.0,"POLYGON ((-1.53722 11.02806, -1.51972 11.02778...",0.181196,0.373418,0.181196,0.373418
108,GHA,788.0,"POLYGON ((-2.73492 11.00012, -2.73444 10.98945...",-0.000045,0.217887,-0.000045,0.217887


In [9]:
for col in gha.columns[3:]:
    gha[col] = np.round(gha[col], 2)

In [10]:
titles = ['upward IM\nC-M gap', 'upward IM\nC-T gap', 'upward IM\nC-M gap', 'upward IM\nC-T gap']
for idvar, variable in enumerate(['gap_cm', 'gap_ct', 'gap_cm_min10', 'gap_ct_min10']):
    title = titles[idvar]
    
    if pd.isnull(gha[variable]).sum() > 0:
        missing_kwds = dict(color='white', label='no Data')
    else:
        missing_kwds = None

    f, ax = plt.subplots(1,1, figsize=(10, 10))
    gha.plot(ax=ax, 
            column=variable,
            scheme='user_defined',
            classification_kwds={'bins':[-0.8,-0.6,-0.4,-0.2,0,0.2,0.4,0.6,0.8,1]},
            cmap = 'RdBu',
            edgecolor='k',
            linewidth=0.1,
            legend=True, 
            missing_kwds=missing_kwds)
    
    ax.axis('off')
    
    leg = ax.get_legend()
    leg.set_bbox_to_anchor((-0.1, 0.75, 0.2, 0.2))
    for txt in leg.texts:
        txti = txt.get_text().replace('-0.88', '-1.00').replace('-inf', '-1.00').replace('-0.94', '-1.00')
        txti = re.sub('\.([0-9])0', r'.\1',  txti.replace(', ', ' - '))
        txt.set_text(txti)
    leg.set_title(title, prop={'size': 12, 'weight': 'bold'})
    
    f.savefig(outdir + f'/_4_heatmap_gha_{variable}.pdf', bbox_inches='tight')
    plt.close(f)